In [24]:
!pip install keras
!pip install tensorflow

  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=237761c3af0f786b72be8bf82a0f601c8d273086692652c91617392aeb901599
  Stored in directory: c:\users\coyol\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.0
    Uninstalling importlib-metadata-3.10.0:
      Successfully uninstalled importlib-metadata-3.10.0


In [26]:
# Implementacion de LSTM
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [10]:
# Se juntan todas las bd
bds = ["bdada","bdavax","bdbnb","bdbtc","bddoge",
       "bddot","bdeth","bdshib","bdsol","bdxrp"]

# Se junta toda la informacion
allData = pd.DataFrame({"OpenTime":[],"Open":[],"High":[],"Low":[],
                         "Close":[],"Volume":[],"CloseTime":[],"QuoteAssetVolume":[],
                         "NumberTrades":[],"TakerBuyBase":[],"TakerQuoteBase":[],
                         "Ignore":[]})
for i in bds:
    DFrame = pd.read_parquet(i+".parquet")
    DFrame["Coin"] = i
    allData = allData.append(DFrame)
    
    

,OpenTime,Open,High,Low,Close,Volume,CloseTime,QuoteAssetVolume,NumberTrades,TakerBuyBase,TakerQuoteBase,Ignore,Coin
0,1.606545e+12,0.14086000,0.14098000,0.14066000,0.14098000,160226.80000000,1.606545e+12,22559.26010100,55.0,63917.70000000,9000.35653900,0,bdada
1,1.606545e+12,0.14099000,0.14128000,0.14099000,0.14109000,512582.80000000,1.606545e+12,72343.05321500,150.0,361426.90000000,51007.28173200,0,bdada
2,1.606545e+12,0.14103000,0.14131000,0.14090000,0.14129000,365651.00000000,1.606545e+12,51591.96907300,89.0,284673.50000000,40164.51992000,0,bdada
3,1.606545e+12,0.14130000,0.14139000,0.14108000,0.14121000,175758.50000000,1.606545e+12,24832.27096800,74.0,81112.30000000,11460.62592500,0,bdada
4,1.606545e+12,0.14119000,0.14126000,0.14119000,0.14121000,48960.00000000,1.606545e+12,6913.77613500,16.0,13865.60000000,1958.24733800,0,bdada
...,...,...,...,...,...,...,...,...,...,...,...,...,...
517063,1.637650e+12,1.04140000,1.04210000,1.04130000,1.04200000,129110.00000000,1.637650e+12,134480.45890000,176.0,69566.00000000,72454.75790000,0,bdxrp
517064,1.637650e+12,1.04200000,1.04200000,1.04100000,1.04150000,115148.00000000,1.637650e+12,119912.26100000,204.0,38642.00000000,40240.43610000,0,bdxrp
517065,1.637650e+12,1.04150000,1.04220000,1.04130000,1.04160000,112157.00000000,1.637650e+12,116833.45260000,269.0,64603.00000000,67297.40970000,0,bdxrp
517066,1.637650e+12,1.04160000,1.04230000,1.04100000,1.04210000,117424.00000000,1.637650e+12,122294.30550000,205.0,72420.00000000,75417.81930000,0,bdxrp


In [33]:
# Reordenamiento de los datos
def reorder(data, window):
    MatX = list()
    VecY = list()
    # Se reordena la informacion
    centi=1
    iniP = 0
    while (centi==1):
        # Nuevas particiones
        finP = iniP + window
        
        # Se revisa que no sea mayor que el maximo de elementos de data
        if finP> len(data)-1:
            centi=0
            break
        else:
            MatX.append(data[iniP:finP])
            VecY.append(data[finP])
            iniP += 1
            
    return np.array(MatX), np.array(VecY)        

In [46]:
npArr = np.array(pd.to_numeric((allData[allData["Coin"]=="bdada"]["High"])))
window = 3
feat = 1
X, y = reorder(npArr,3)
X = X.reshape((X.shape[0],X.shape[1],feat))
X

array([[[0.14098],
        [0.14128],
        [0.14131]],

       [[0.14128],
        [0.14131],
        [0.14139]],

       [[0.14131],
        [0.14139],
        [0.14126]],

       ...,

       [[1.796  ],
        [1.795  ],
        [1.796  ]],

       [[1.795  ],
        [1.796  ],
        [1.796  ]],

       [[1.796  ],
        [1.796  ],
        [1.796  ]]])

In [ ]:
# Modelo
neuronas = 50
modelo = Sequential()
modelo.add(LSTM(units=neuronas, activation='relu', input_shape=(window, feat)))
modelo.add(Dense(1))
modelo.compile(optimizer='adam', loss='mse')

# Estimacion
modelo.fit(X, y, epochs=300, verbose=0)

